In [ ]:
%load_ext autoreload
%autoreload 2

import sys
if '..' not in sys.path:
    sys.path.append('..')

In [180]:
import conllu

with open('../data/ud/en_ewt-ud-train.conllu', 'r') as f:
    sentences = conllu.parse(f.read())

In [196]:
deps = set()

for sent in sentences:
    for tok in sent:
        deps.add(tok['deprel'])
        
print(deps)
        

{'xcomp', 'list', 'advcl:relcl', 'vocative', 'det:predet', 'dep', 'obl:tmod', 'iobj', 'advmod', 'appos', 'nmod:tmod', 'acl:relcl', 'nmod', 'case', '_', 'conj', 'acl', 'goeswith', 'nsubj', 'ccomp', 'nmod:npmod', 'root', 'nmod:desc', 'obl:npmod', 'cop', 'parataxis', 'aux', 'csubj', 'csubj:pass', 'det', 'compound:prt', 'discourse', 'mark', 'expl', 'advcl', 'cc', 'reparandum', 'csubj:outer', 'nummod', 'nsubj:pass', 'cc:preconj', 'dislocated', 'fixed', 'aux:pass', 'obl:agent', 'nsubj:outer', 'amod', 'obj', 'nmod:poss', 'orphan', 'obl', 'punct', 'compound', 'flat'}


In [199]:
deps

{'_',
 'acl',
 'acl:relcl',
 'advcl',
 'advcl:relcl',
 'advmod',
 'amod',
 'appos',
 'aux',
 'aux:pass',
 'case',
 'cc',
 'cc:preconj',
 'ccomp',
 'compound',
 'compound:prt',
 'conj',
 'cop',
 'csubj',
 'csubj:outer',
 'csubj:pass',
 'dep',
 'det',
 'det:predet',
 'discourse',
 'dislocated',
 'expl',
 'fixed',
 'flat',
 'goeswith',
 'iobj',
 'list',
 'mark',
 'nmod',
 'nmod:desc',
 'nmod:npmod',
 'nmod:poss',
 'nmod:tmod',
 'nsubj',
 'nsubj:outer',
 'nsubj:pass',
 'nummod',
 'obj',
 'obl',
 'obl:agent',
 'obl:npmod',
 'obl:tmod',
 'orphan',
 'parataxis',
 'punct',
 'reparandum',
 'root',
 'vocative',
 'xcomp'}

In [ ]:
hierarchy = {
    'NOUN': ['NOUN', 'PROPN'],
    'VERB': ['VERB', 'AUX'],
    'MOD': ['ADJ', 'ADV'],
    'FUNC': ['DET', 'PRON', 'ADP', 'CCONJ', 'SCONJ', 'PART'],
    'NUM': ['NUM'],
    'OTHER': ['INTJ', 'SYM', 'PUNCT', 'X', '_']
}

to_parent = {}
for k, vs in hierarchy.items():
    to_parent[k] = k
    for v in vs:
        to_parent[v] = k
        
to_parent

In [6]:
m_sentences = []
for sent in sentences:
    cur = []
    for tok in sent:
        cur.append((tok['form'], to_parent[tok['upostag']]))
    m_sentences.append(cur)

In [36]:
import random
from nltk.tokenize.treebank import TreebankWordDetokenizer

detok = TreebankWordDetokenizer()

def noise(sent, init=0.2):
    traj = [detok.detokenize([s[0] for s in sent])]
    
    for drop in ['OTHER', 'MOD', 'FUNC', 'NUM', 'VERB', 'NOUN']:
        new_sent = []
        for tok, pos in sent:
            if pos != drop: new_sent.append((tok, pos))
        sent = new_sent
        traj.append(detok.detokenize([s[0] for s in sent]))
        
    return traj

In [160]:
import numpy as np

samples = np.random.choice([1,2,3], 2, replace=False, p=[1/3,1/3,1/3])

In [162]:
set(samples)

{2, 3}

In [177]:
import numpy as np

importance = {
    'OTHER': 1.5,
    'FUNC': 2,
    'NUM': 4,
    'MOD': 5,
    'VERB': 8,
    'NOUN': 9
}

def noise(sent, traj_length=6):
    traj = [detok.detokenize([s[0] for s in sent])]
   
    for i in range(traj_length):
        if not sent: break
        N = max(1, len(sent) // (traj_length - i))
        
        weights = [1 / importance[pos] for _, pos in sent]
        tot = sum(weights)
        weights = [w / tot for w in weights]
        
        to_drop = np.random.choice(
            range(len(sent)),
            min(N, len(sent)),
            p=weights,
            replace=False
        )

        sent  = [tok for j, tok in enumerate(sent) if j not in to_drop]
        traj.append(detok.detokenize(s[0] for s in sent))
        
    if traj[-1] != '': traj.append('')
    return traj[::-1]

In [179]:
noise(m_sentences[10])

['',
 'killed member Party kidnapping',
 'Zaman killed member Kurdistan Democratic Party kidnapping',
 'Zaman Guerrillas killed member the Kurdistan Democratic Party kidnapping Mosul',
 'Al Zaman Guerrillas killed member the Kurdistan Democratic Party after kidnapping Mosul.',
 'Al Zaman: Guerrillas killed a member the Kurdistan Democratic Party after kidnapping in Mosul.',
 'Al - Zaman: Guerrillas killed a member of the Kurdistan Democratic Party after kidnapping him in Mosul.']

In [27]:
!python ../dep.py ../data/ud/en_ewt-ud-train.conllu ../data/ud_train.jsonl --redundant=1 --weight=0.1

^C
Traceback (most recent call last):
  File "../dep.py", line 106, in <module>
    main()
  File "../dep.py", line 84, in main
    sentences = conllu.parse(f.read())
  File "/usr/local/anaconda3/envs/evo/lib/python3.8/site-packages/conllu/__init__.py", line 19, in parse
    return SentenceList(parse_incr(
  File "/usr/local/anaconda3/envs/evo/lib/python3.8/site-packages/conllu/models.py", line 303, in __init__
    sentences = list(sentences)
  File "/usr/local/anaconda3/envs/evo/lib/python3.8/site-packages/conllu/__init__.py", line 39, in generator
    yield parse_token_and_metadata(
  File "/usr/local/anaconda3/envs/evo/lib/python3.8/site-packages/conllu/parser.py", line 96, in parse_token_and_metadata
    tokens.append(parse_line(line, fields, field_parsers))
  File "/usr/local/anaconda3/envs/evo/lib/python3.8/site-packages/conllu/parser.py", line 131, in parse_line
    value = field_parsers[field](line_split, i)
  File "/usr/local/anaconda3/envs/evo/lib/python3.8/site-packages/conl

In [ ]:
!python ../dep.py ../data/ud/en_ewt-ud-dev.conllu ../data/ud_dev.jsonl --redundant=3 --weight=0.1

In [ ]:
from data import TrajectoryDataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = TrajectoryDataset.from_disk(
    path='../data/ud/ud_train.jsonl',
    max_len=64,
    tokenizer=tokenizer
)

eval_dataset = TrajectoryDataset.from_disk(
    path='../data/ud/ud_dev.jsonl',
    max_len=64,
    tokenizer=tokenizer,
    limit=100
)

In [ ]:
from torch.utils.data import DataLoader
from data import StratifiedInfiniteSampler

train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    sampler=StratifiedInfiniteSampler(train_dataset, 2),
    collate_fn=lambda x: zip(*x)
)

eval_loader = DataLoader(
    eval_dataset,
    batch_size=1,
    shuffle=True
)

In [ ]:
from model import Evolver
from torch.optim import AdamW

evolver = Evolver(
    d_model=512,
    nhead=8,
    max_len=64,
    encoder_layers=6,
    decoder_layers=6,
    device='cpu'
)

optim = AdamW(evolver.parameters(), lr=3e-4)

In [ ]:
from train import train_evolver

train_evolver(
    evolver, optim, None,
    train_loader, eval_loader,
    train_steps=1,
    grad_accum_steps=1,
    checkpoint_at=2,
    eval_at=1,
    num_particles=5,
    threshold=2,
    temperature=1.0,
    device='cpu',
    prefix='test-local'
)

In [ ]:
from train import evaluate_evolver

evaluate_evolver(evolver, eval_loader, 'cpu')

In [ ]:
!python ../train.py \
    --train ../data/ud/ud.jsonl \
    --eval ../data/ud/en_ewt-ud-dev.conllu \
    --config ../configs/ud.json \
    --prefix ud-1.0.0 \
    --device cpu

In [22]:
from data import Seq2SeqDataset
from transformers import BertTokenizer

dataset = Seq2SeqDataset.from_trajectories(
    '../data/ud/ud_train.jsonl',
    denoising=False,
    max_len=64,
    tokenizer=tokenizer
)

tokenizing input/output pairs...
done in 4.01 seconds!


In [42]:
from torch.utils.data import DataLoader
from data import StratifiedInfiniteSampler

train_loader = DataLoader(
    dataset,
    batch_size=128,
    sampler=StratifiedInfiniteSampler(dataset, 128),
)

# just to test
eval_loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True
)